In [2]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
def get_train_data(location):
    print(location)
#         X = np.load(os.path.join(location, 'training.npz'))['xtrain']
#         y=np.load(os.path.join(location, 'training.npz'))['ytrain']
    df=pd.read_csv( 'salary.csv')
    X,y=df.iloc[:,0],df.iloc[:,1]
    X=np.array(X).astype('float32').reshape(-1,1)
    y=np.array(y/100000).astype('float32').reshape(-1,1)
    return X,y

In [ ]:
sess=sagemaker.Session()


In [ ]:
import boto3

In [5]:
model=tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=False,pooling='avg',weights='imagenet',input_shape=(128,128,3))

In [3]:
tf.__version__

'2.3.1'

In [ ]:
role='arn:aws:iam::032934527328:role/service-role/AmazonSageMaker-ExecutionRole-20201014T214516'

In [ ]:
hyperparameters={'epochs':300}

In [ ]:
estimator=TensorFlow(
    entry_point= "custom_train.py",             # Your entry script
    role=role,
    framework_version="2.0.0",               # TensorFlow's version
    py_version='py3',
    train_instance_count=1,                   # "The number of GPUs instances to use"
#     train_instance_type='ml.c5.2xlarge',
    train_instance_type='local',
    hyperparameters=hyperparameters,
)

In [ ]:
prefix = 'salaryestimator'

In [ ]:
# !pip install 'sagemaker[local]'

In [ ]:
training_input_path   = sess.upload_data('salary.csv', key_prefix=prefix+'/training_csv')
# training_input_path='file://salary.csv'
estimator.fit({'train':training_input_path})

In [ ]:
predictor=estimator.deploy(initial_instance_count=1,instance_type='local')

In [ ]:
yhat=predictor.predict(get_train_data('salary.csv')[0])
yhat=np.array(yhat['predictions'])

In [ ]:
plt.plot(*get_train_data('salary.csv'))
plt.plot(get_train_data('salary.csv')[0],yhat)

Able to verify weights are correct, not sure why predictions are off from the deployed sagemkaer model
predictions work when using local keras model below

In [ ]:
# download model from s3 and unzip
downloadedmodel=tf.keras.models.load_model(
    '/Users/ashingote/Downloads/000000001', custom_objects=None, compile=True
)

In [ ]:
yhat=downloadedmodel.predict(get_train_data('salary.csv')[0])

In [ ]:
plt.plot(*get_train_data('salary.csv'))
plt.plot(get_train_data('salary.csv')[0],yhat)